In [86]:
from PIL import Image
import numpy as np
import random

maze_image = Image.open('5x5.png')




In [87]:
#Converts image to a defaut tuple of RBG .convert() or greyscale .convert("L") or a Palletescale .convert("P")
# Greyscale averages the RBG tuple into a single number so its easier to work with
maze_image = maze_image.convert('P')

In [88]:
#resize image to fit scale of matrix
#Helps to convert some random scale of the image into a scale we can use
maze_image = maze_image.resize((5, 5))

In [89]:
#converts image to array

image_array = np.array(maze_image)

print(image_array)


[[1 1 0 0 2]
 [1 0 0 1 0]
 [0 0 0 1 0]
 [0 1 0 1 0]
 [3 1 0 0 0]]


In [90]:
#image_array = np.pad(image_array, pad_width=1, mode='constant', constant_values=1)
#print(image_array)

In [91]:
states = 25
actions = 4
state_action_table = np.zeros((states, actions))
print(state_action_table)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [92]:
def create_rewards_matrix(maze):
    rewards = np.full_like(maze, -1, dtype=int)
    for y, row in enumerate(maze):
        for x, cell in enumerate(row):
            if cell == 3:  # Charging station
                rewards[y][x] = 500
            elif cell == 1:  # Wall
                rewards[y][x] = -100
            elif cell == 2:  # Charging station
                rewards[y][x] = 0
    return rewards

x = create_rewards_matrix(image_array)
print(x)

[[-100 -100   -1   -1    0]
 [-100   -1   -1 -100   -1]
 [  -1   -1   -1 -100   -1]
 [  -1 -100   -1 -100   -1]
 [ 500 -100   -1   -1   -1]]


In [93]:
start_state = np.where(image_array == 2)
row_index = start_state[0][0]
column_index = start_state[1][0]
start_state = (row_index, column_index)
print(start_state)

# Set the parameters for the Q-learning algorithm
num_episodes = 100
alpha = 0.8
gamma = 0.95
epsilon = 0.1

(0, 4)


In [94]:

def Qlearning(matrix, q_table, start_state, num_episodes, alpha, gamma, epsilon):
    # Get the number of states and actions from the Q-table
    num_states, num_actions = q_table.shape
    
    # For each episode
    for episode in range(num_episodes):
        # Reset the environment and set the starting state
        i, j = start_state
        state = i * num_states + j
        # Loop over time steps within the episode
        while True:
            # Choose an action using an epsilon-greedy policy
            if np.random.uniform() < epsilon:
                # Choose a random action
                action = np.random.choice(len(matrix[state]))
            else:
                # Choose the action with the highest Q-value for the current state
                action = np.argmax(q_table[state])
            # Take the chosen action and observe the next state and reward
            i, j = divmod(action, num_states)
            next_state = i * num_states + j
            reward = matrix[i, j]
            # Update the Q-value for the current state and action using the Q-learning update rule
            q_table[state][action] += alpha * (reward + gamma * np.max(q_table[next_state]) - q_table[state][action])
            # Update the state
            state = next_state
            # If the episode has ended, break out of the time step loop
            if reward != 0:
                break
    return q_table


In [95]:
updatedQtable = Qlearning(image_array, state_action_table, start_state, num_episodes, alpha, gamma, epsilon)
print(updatedQtable)

IndexError: index 4 is out of bounds for axis 0 with size 4